# Environment setup

In [1]:
from sys import platform
from analysis_utils import *

/local_disk/abhinav/eye-blink-music/4-blink-train-distance/analysis_utils.py:407: SyntaxWarning: invalid escape sequence '\l'
  axp.set_ylabel("$\lambda_i$", color=bars[0].get_facecolor(), alpha=1)
/local_disk/abhinav/eye-blink-music/4-blink-train-distance/analysis_utils.py:531: SyntaxWarning: invalid escape sequence '\i'
  print(f'{round(accuracy, 2)}%\t{true_positive}/{len(blink_trains)}\tq={round(q_Hz, 2)}/s\tt=$\infty$ s\n{cm}')


In [2]:
# Path to read trial-merged FIF files from:
MERGED_FIF_PATH = os.path.join('..', '3-find-blinks', '3.2-manually-select-blinker-blinks', 'merged-fif')

In [3]:
# Path to export calculated VP-distances to:
VP_NPY_PATH = 'npy'

# Victor-Purpura Distance
See Elephant's documentation:

https://elephant.readthedocs.io/en/v0.9.0/reference/toctree/spike_train_dissimilarity/elephant.spike_train_dissimilarity.victor_purpura_distance.html

Some helpful figures are also available from this SfN'03 poster:

http://www-users.med.cornell.edu/~jdvicto/vps/sfn03_vimere.pdf

In [4]:
subjects = [
    1, 
    #2, # too few blinks
    3, 4, 
    #5, # too few blinks
    6, 7, 8,
]

In [5]:
event_dict = {
    "Listening/chor-038": 1,
    "Listening/chor-096": 2,
    "Listening/chor-101": 3,
    "Listening/chor-019": 4,
    "Imagery/chor-038": 11,
    "Imagery/chor-096": 12,
    "Imagery/chor-101": 13,
    "Imagery/chor-019": 14,
}

# VP-Distance on trial-merged FIFs

In [6]:
RAWS_SUB = read_raws_by_subject(MERGED_FIF_PATH, SUBJECT='*')
RAWS_SUB

Found ['../3-find-blinks/3.2-manually-select-blinker-blinks/merged-fif/subject-1_blinks_merged_raw.fif', '../3-find-blinks/3.2-manually-select-blinker-blinks/merged-fif/subject-3_blinks_merged_raw.fif', '../3-find-blinks/3.2-manually-select-blinker-blinks/merged-fif/subject-4_blinks_merged_raw.fif', '../3-find-blinks/3.2-manually-select-blinker-blinks/merged-fif/subject-5_blinks_merged_raw.fif', '../3-find-blinks/3.2-manually-select-blinker-blinks/merged-fif/subject-6_blinks_merged_raw.fif', '../3-find-blinks/3.2-manually-select-blinker-blinks/merged-fif/subject-7_blinks_merged_raw.fif', '../3-find-blinks/3.2-manually-select-blinker-blinks/merged-fif/subject-8_blinks_merged_raw.fif']
Opening raw data file ../3-find-blinks/3.2-manually-select-blinker-blinks/merged-fif/subject-1_blinks_merged_raw.fif...
    Range : 0 ... 158663 =      0.000 ...  2479.109 secs
Ready.
Reading 0 ... 158663  =      0.000 ...  2479.109 secs...
Opening raw data file ../3-find-blinks/3.2-manually-select-blinker

{1: <Raw | subject-1_blinks_merged_raw.fif, 72 x 158664 (2479.1 s), ~87.2 MiB, data loaded>,
 3: <Raw | subject-3_blinks_merged_raw.fif, 72 x 158664 (2479.1 s), ~87.2 MiB, data loaded>,
 4: <Raw | subject-4_blinks_merged_raw.fif, 72 x 158664 (2479.1 s), ~87.2 MiB, data loaded>,
 5: <Raw | subject-5_blinks_merged_raw.fif, 72 x 158664 (2479.1 s), ~87.2 MiB, data loaded>,
 6: <Raw | subject-6_blinks_merged_raw.fif, 72 x 158664 (2479.1 s), ~87.2 MiB, data loaded>,
 7: <Raw | subject-7_blinks_merged_raw.fif, 72 x 158664 (2479.1 s), ~87.2 MiB, data loaded>,
 8: <Raw | subject-8_blinks_merged_raw.fif, 72 x 158664 (2479.1 s), ~87.2 MiB, data loaded>}

Process VP distance for read FIFs:

In [7]:
SAVE_RESULTS = True

nmax = 1802 # 1803rd sample
bpm = 100
bps = 100/60
qtr_note_dur = 1/bps

for SUBJECT in subjects:
    
    raw = RAWS_SUB[SUBJECT]
    tmax = nmax/raw.info['sfreq']
    epochs, order = trials_as_epochs(
        raw, 
        stim_channel = 'trials',
        event_dict = event_dict,
        picks = raw.ch_names[-8::], # excludes EEG channels
        tmin = 0, # first sample
        tmax = tmax, 
    )
    # order here is the sorting order for epochs by event_dict IDs (1, 2, 3, 4, 11, 12, 13, 14)

    tau_list = np.arange(start=0, stop=tmax, step=qtr_note_dur)
    tau_list = np.concatenate(
        (tau_list, [np.inf]) # also run for q=inf
    )

    results_listen = sweep_tau(
        epochs,
        order, 
        include_events = [1, 2, 3, 4],
        tau_list = tau_list,
    )
    if SAVE_RESULTS: np.save(
        os.path.join(VP_NPY_PATH, f'sub{SUBJECT}_results_listen_taus-{len(tau_list)}'), 
        results_listen)

    results_imagery = sweep_tau(
        epochs,
        order, 
        include_events = [11, 12, 13, 14],
        tau_list = tau_list,
    )
    if SAVE_RESULTS: np.save(
        os.path.join(VP_NPY_PATH, f'sub{SUBJECT}_results_imagery_taus-{len(tau_list)}'), 
        results_imagery)

Finding events on: trials
88 events found on stim channel trials
Event IDs: [ 1  2  3  4 11 12 13 14]
['midi', 'expectation', 'tactile_cue', 'envelope', 'ica_1', 'blinks', 'sweeps', 'trials']
Not setting metadata
88 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 88 events and 1803 original time points ...
0 bad epochs dropped
25.0%	11/44	q=inf/s	t=0.0s
[[ 0  0  0 11]
 [ 0  0  0 11]
 [ 0  0  0 11]
 [ 0  0  0 11]]
31.82%	14/44	q=1.67/s	t=0.6s
[[4 2 4 1]
 [4 1 2 4]
 [5 2 3 1]
 [0 2 3 6]]
29.55%	13/44	q=0.83/s	t=1.2s
[[6 0 4 1]
 [3 2 2 4]
 [5 1 0 5]
 [0 1 5 5]]
31.82%	14/44	q=0.56/s	t=1.8s
[[6 0 4 1]
 [2 2 5 2]
 [5 1 1 4]
 [2 1 3 5]]
29.55%	13/44	q=0.42/s	t=2.4s
[[6 0 3 2]
 [3 1 4 3]
 [3 2 2 4]
 [4 1 2 4]]
29.55%	13/44	q=0.33/s	t=3.0s
[[5 2 3 1]
 [4 2 3 2]
 [4 1 2 4]
 [2 2 3 4]]
27.27%	12/44	q=0.28/s	t=3.6s
[[5 2 3 1]
 [4 2 3 2]
 [5 2 1 3]
 [2 2 3 4]]
25.0%	11/44	q=0.24/s	t=4.2s
[[4 2 3 2]
 [5 2 2 2]
 [6 2 1 2]
 [2 2 3 4]